In [1]:
from frictionless import Resource, transform, steps, Field, Schema
%cd ..

/opt/st-gallen-urban-indicators


## 1. Import RAW data and aggregate

To work with data from `data/mobility/raw/` we need to import and aggregate the data first.

```sh
$ tree data/mobility/raw/
data/mobility/raw/
├── insight_16312_inward_start_2021-06-14_end_2021-06-20_all.xlsx
├── insight_16312_inward_start_2021-06-14_end_2021-06-20_nonregular.xlsx
├── insight_16312_inward_start_2021-06-14_end_2021-06-20_regular.xlsx
├── insight_16312_inward_total_start_2021-06-14_end_2021-06-20_all.xlsx
├── insight_16312_inward_total_start_2021-06-14_end_2021-06-20_nonregular.xlsx
├── insight_16312_inward_total_start_2021-06-14_end_2021-06-20_regular.xlsx
├── insight_16312_outward_start_2021-06-14_end_2021-06-20_all.xlsx
├── insight_16312_outward_start_2021-06-14_end_2021-06-20_nonregular.xlsx
├── insight_16312_outward_start_2021-06-14_end_2021-06-20_regular.xlsx
├── insight_16312_outward_total_start_2021-06-14_end_2021-06-20_all.xlsx
├── insight_16312_outward_total_start_2021-06-14_end_2021-06-20_nonregular.xlsx
├── insight_16312_outward_total_start_2021-06-14_end_2021-06-20_regular.xlsx
├── insight_16312_overview_reason_start_2021-06-14_end_2021-06-20.xlsx
└── insight_16312_overview_trip_type_start_2021-06-14_end_2021-06-20.xlsx
```

Each file has three columns, either Origin (for inward trips) or Destination (for outward trips)

In [36]:
!frictionless describe "data/mobility/raw/insight_16312_inward_start_2021-06-14_end_2021-06-20_all.xlsx" --yaml

/opt/st-gallen-urban-indicators/venv/lib64/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
encoding: utf-8
format: xlsx
hashing: md5
name: insight_16312_inward_start_2021-06-14_end_2021-06-20_all
path: data/mobility/raw/insight_16312_inward_start_2021-06-14_end_2021-06-20_all.xlsx
profile: tabular-data-resource
schema:
  fields:
    - name: Origin ID
      type: integer
    - name: Origin
      type: integer
    - name: Count
      type: integer
scheme: file


Hence we will define a schema for both types of tables renaming the first two columns as ref_id and ref during import.

In [31]:
miss_schema_od = Schema(fields=[
    Field(name="ref_id", type="string"),
    Field(name="ref", type="string"),
    Field(name="count", type="integer"),
])

In [37]:
resources = []
for w in [{"start":"2021-06-14","end":"2021-06-20"}]:
    for d in ["inward","outward"]:
        for r in ["all","regular","nonregular"]:
            resources.append(transform( 
                Resource(
                    path=f"data/mobility/raw/insight_16312_{d}_start_{w['start']}_end_{w['end']}_{r}.xlsx",
                    schema=miss_schema_od  # use custom schema defined above
                ),
                steps=[
                    steps.table_normalize(),
                    steps.row_filter(formula="ref_id != 'Other' and ref_id != 'Total'"),  # discard last two columns showing totals and other
                    steps.field_add(name="week_start", value=w['start']),
                    steps.field_add(name="week_end", value=w['end']),
                    steps.field_add(name="direction", value=d),
                    steps.field_add(name="reason", value=r),
                    steps.table_normalize(),
                ]
            ))

aggregated = resources[0]
resources.pop(0)

for i, d in enumerate(resources):
    aggregated = transform(
        aggregated,
        steps=[
            steps.table_merge(resource=resources[i], sort_by_field="direction"),
            steps.table_write(path="data/mobility/od-mobility.csv")
        ]
    )

aggregated.to_petl()

/opt/st-gallen-urban-indicators/venv/lib64/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


ref_id,ref,count,week_start,week_end,direction,reason
9000,9000,1211,2021-06-14,2021-06-20,inward,all
9008,9008,771,2021-06-14,2021-06-20,inward,all
9016,9016,301,2021-06-14,2021-06-20,inward,all
9010,9010,162,2021-06-14,2021-06-20,inward,all
9015,9015,116,2021-06-14,2021-06-20,inward,all


In [30]:
pivoted = transform(
    aggregated,
    steps=[
        steps.field_add(name="direction_reason", function=lambda x: f"{x['direction']}_{x['reason']}"),
        steps.field_remove(names=["ref","week_start","week_end","direction","reason"]),
        steps.table_pivot(f1="ref_id", f2="direction_reason", f3="count", aggfun=sum),
        steps.table_write(path="data/mobility/od-mobility-matrix.csv"),
    ]
)

pivoted.to_petl().display(limit=100)

ref_id,inward_all,inward_nonregular,inward_regular,outward_all,outward_nonregular,outward_regular
9000,1211,1033,178,1186,1030,155
9008,771,683,88,798,732,65
9010,162,147,None,126,118,None
9011,48,44,None,56,50,None
9014,89,73,None,71,59,None
9015,116,90,26,111,79,31
9016,301,269,32,307,276,31
9030,24,24,None,41,34,None
9037,24,22,None,None,None,None
9042,28,24,None,44,35,None


## 2. Create a snapshots

### 2.1 Footprint for incoming trips

In [107]:
# helpers

def category_mapping(value, mappings):
    if len(mappings) > 0 and value:
        for mapping in mappings:
            if "from" in mapping.keys() and "to" in mapping.keys() and "cat" in mapping.keys():
                if value >= mapping["from"] and value < mapping["to"]:
                    return mapping["cat"]
    return None

def color_mapping(value, mapping, key):
    if value in mapping.keys():
        if key in mapping[value].keys():
            return mapping[value][key]
    return None

colors = {
    "a": { "fill": "#0028b8", "fill-opacity": 0.5, "stroke": "#ffffff", "stroke-width": 0.5, "stroke-opacity": 0.9 },
    "b": { "fill": "#455db5", "fill-opacity": 0.5, "stroke": "#ffffff", "stroke-width": 0.5, "stroke-opacity": 0.9 },
    "c": { "fill": "#8a93b2", "fill-opacity": 0.5, "stroke": "#ffffff", "stroke-width": 0.5, "stroke-opacity": 0.9 },
    "d": { "fill": "#bfa98f", "fill-opacity": 0.5, "stroke": "#ffffff", "stroke-width": 0.5, "stroke-opacity": 0.9 },
    "e": { "fill": "#df9848", "fill-opacity": 0.5, "stroke": "#ffffff", "stroke-width": 0.5, "stroke-opacity": 0.9 },
    "f": { "fill": "#ff8800", "fill-opacity": 0.5, "stroke": "#ffffff", "stroke-width": 0.5, "stroke-opacity": 0.9 },
}

In [81]:
geoshape = transform(
    Resource(
        path="data/misc/postalcodes.geojson"
    ),
    steps=[
        steps.field_remove(names=["UUID","OS_UUID","STATUS","INAEND","ZUSZIFF"]),
        steps.field_update(name="PLZ", new_name="ref_id"),
        steps.table_normalize(),
    ]
)

geoshape.infer()
print(geoshape.schema)

{'fields': [{'name': 'ref_id', 'type': 'integer'}, {'name': '_geom', 'type': 'string'}]}


In [111]:
from frictionless import Resource, Package, transform, steps
import json

data = Resource(
    "data/mobility/od-mobility-matrix.resource.yaml",
)

categories = [
    { "from": 0, "to": 150, "cat": "a"},
    { "from": 150, "to": 300, "cat": "b"},
    { "from": 300, "to": 450, "cat": "c"},
    { "from": 450, "to": 600, "cat": "d"},
    { "from": 600, "to": 750, "cat": "e"},
    { "from": 750, "to": 1500, "cat": "f"},
]

for s in ["inward_all", "inward_regular", "inward_nonregular", "outward_all", "outward_regular", "outward_nonregular"]:

    pkg = Package(
        name=f"10-{s}",
        title=f"{s}"
    )

    styled = transform(
        data,
        steps=[
            steps.table_normalize(),
            steps.field_add(name="title", function= lambda x: f"Postleitzahlgebiet {x['ref_id']}"),
            steps.field_add(name="description", 
                function=lambda x: f"ca. {x['inward_all']} Fahrten/Woche kommend aus {x['ref_id']}<br>ca. {x['inward_regular']} regelmässige Fahrten/Woche kommend aus {x['ref_id']}<br>ca. {x['inward_nonregular']} Fahrten/Woche kommend aus {x['ref_id']}<br>ca. {x['outward_all']} Fahrten/Woche gehend nach {x['ref_id']}<br>ca. {x['outward_regular']} regelmässige Fahrten/Woche gehend nach {x['ref_id']}<br>ca. {x['outward_nonregular']} unregelmässige Fahrten/Woche gehend nach {x['ref_id']}<br>"
            ),
            steps.field_add(name="cat", type="string", function=lambda x: category_mapping(x[s], categories)),
            steps.row_filter(formula="cat is not None"),
            steps.field_add(name="fill", type="string", function=lambda x: color_mapping(x["cat"], colors, "fill")),
            steps.field_add(name="fill-opacity", type="number", function=lambda x: color_mapping(x["cat"], colors, "fill-opacity")),
            steps.field_add(name="stroke", type="string", function=lambda x: color_mapping(x["cat"], colors, "stroke")),
            steps.field_add(name="stroke-width", type="number", function=lambda x: color_mapping(x["cat"], colors, "stroke-width")),
            steps.field_add(name="stroke-opacity", type="number", function=lambda x: color_mapping(x["cat"], colors, "stroke-opacity")),
            # steps.field_update(name="wkt", new_name="_geom"),
            steps.table_normalize(),
            steps.table_join(resource=geoshape, field_name="ref_id", mode="inner"),
        ]
    )

    geojson = styled.write(path="tmp/styled.geojson")

    with open("tmp/styled.geojson") as gjson:
        pkg.add_resource(
            Resource(
                name="data",
                mediatype="application/geo+json",
                data=json.load(gjson)
            )
        )
    
    pkg.add_resource(
        dict(
            name="mapbox-background",
            path = "mapbox://styles/gemeindescan/ckc4sha4310d21iszp8ri17u2",
            mediatype = "application/vnd.mapbox-vector-tile",
        )
    )

    pkg.sources = [
        {
            "url": "https://mip.swisscom.ch",
            "title": "Swisscom Mobility Insights",
        },
        {
            "title": "Karte: Mapbox, © OpenStreetMap",
            "url": "https://www.openstreetmap.org/copyright"
        },
    ]

    pkg["views"] = [
            {
                "name": "mapview",
                "resources": [
                    "mapbox-background",
                    "data",
                ],
                "spec": {
                    "attribution": "",
                    "bounds": [
                        "geo:47.41525506820663,9.352516279706524",
                        "geo:47.43317355684985,9.396247605270359"
                    ],
                    "title": "Fussabdruck Bach Areal 14.-20. Juni 2021",
                    "description": "",
                    "legend": [
                        {
                            "fillColor": "#0028b8",
                            "fillOpacity": 0.8,
                            "strokeColor": "#ffffff",
                            "strokeOpacity": 0.9,
                            "strokeWidth": 1.5,
                            "size": 1,
                            "shape": "circle",
                            "primary": True,
                            "label": "21-149 Fahrten/Woche",
                        },
                        {
                            "fillColor": "#455db5",
                            "fillOpacity": 0.8,
                            "strokeColor": "#ffffff",
                            "strokeOpacity": 0.9,
                            "strokeWidth": 1.5,
                            "size": 1,
                            "shape": "circle",
                            "primary": True,
                            "label": "150-299 Fahrten/Woche",
                        },
                        {
                            "fillColor": "#8a93b2",
                            "fillOpacity": 0.8,
                            "strokeColor": "#ffffff",
                            "strokeOpacity": 0.9,
                            "strokeWidth": 1.5,
                            "size": 1,
                            "shape": "circle",
                            "primary": True,
                            "label": "300-449 Fahrten/Woche",
                        },
                        {
                            "fillColor": "#bfa98f",
                            "fillOpacity": 0.8,
                            "strokeColor": "#ffffff",
                            "strokeOpacity": 0.9,
                            "strokeWidth": 1.5,
                            "size": 1,
                            "shape": "circle",
                            "primary": True,
                            "label": "450-599 Fahrten/Woche",
                        },
                        {
                            "fillColor": "#df9848",
                            "fillOpacity": 0.8,
                            "strokeColor": "#ffffff",
                            "strokeOpacity": 0.9,
                            "strokeWidth": 1.5,
                            "size": 1,
                            "shape": "circle",
                            "primary": True,
                            "label": "600-749 Fahrten/Woche",
                        },
                        {
                            "fillColor": "#ff8800",
                            "fillOpacity": 0.8,
                            "strokeColor": "#ffffff",
                            "strokeOpacity": 0.9,
                            "strokeWidth": 1.5,
                            "size": 1,
                            "shape": "circle",
                            "primary": True,
                            "label": "750-1500 Fahrten/Woche",
                        }
                    ]
                },
                "specType": "gemeindescanSnapshot"
            }
        ]

    with open(f"snapshots/10-{s}.json", "w") as pkg_file:
        json.dump(pkg, pkg_file, indent=2)

### 2.2. Footprint for outgoing trips

### 2.3. Deltas for incoming and outgoing trips